## Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.

In [14]:
import pandas as pd
import os

### Склеим все данные в одну таблицу

К главному датасету добавим данные из других файлов

In [15]:
df = pd.read_csv('data/D_clients.csv')\
                    .rename(columns={'ID': 'ID_CLIENT'})
df.head()

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,Московская область,1,0
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,Читинская область,0,0
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,Иркутская область,0,1
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,Новосибирская область,Новосибирская область,1,1
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,Красноярский край,Красноярский край,1,0


In [16]:
salary = pd.read_csv('data/D_salary.csv')[['PERSONAL_INCOME', 'ID_CLIENT']]

loan_num = pd.read_csv('data/D_loan.csv').groupby('ID_CLIENT')['ID_LOAN'].count().reset_index()

closed_loan = pd.read_csv('data/D_close_loan.csv')
closed_loan = closed_loan.merge(pd.read_csv('data/D_loan.csv'), on='ID_LOAN', how='outer')\
                         .groupby('ID_CLIENT')['CLOSED_FL'].sum().reset_index()

target = pd.read_csv('data/D_target.csv')

In [17]:
df = df.merge(salary, on='ID_CLIENT', how='outer')\
       .merge(target, on='ID_CLIENT', how='outer')\
       .merge(loan_num, on='ID_CLIENT', how='outer')\
       .merge(closed_loan, on='ID_CLIENT', how='outer')

In [18]:
# Переименовываем название колонок
df = df.rename(columns={'ID_LOAN': 'LOAN_NUM_TOTAL', 'CLOSED_FL': 'LOAN_NUM_CLOSED'})

In [19]:
df = df[['AGREEMENT_RK', 'TARGET', 'AGE', 'SOCSTATUS_WORK_FL',
    'SOCSTATUS_PENS_FL', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS',
    'PERSONAL_INCOME', 'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED']]

Ниже представлен список колонок, которые находятся в итоговом датасете после склейки и агрегации данных

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.

In [20]:
df

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,59910150.0,0.0,49,1,0,1,2,1,5000.0,1.0,1.0
1,59910150.0,0.0,49,1,0,1,2,1,5000.0,1.0,1.0
2,59910230.0,0.0,32,1,0,1,3,3,12000.0,1.0,1.0
3,59910525.0,0.0,52,1,0,1,4,0,9000.0,2.0,1.0
4,59910803.0,0.0,39,1,0,1,1,1,25000.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
16295,NaN,NaN,22,1,0,0,0,0,NaN,NaN,NaN
16296,NaN,NaN,26,1,0,1,1,1,NaN,NaN,NaN
16297,NaN,NaN,34,1,0,0,2,2,NaN,NaN,NaN
16298,NaN,NaN,36,1,0,0,0,0,NaN,NaN,NaN


In [21]:
df.to_csv('data/data.csv', index=False)

In [22]:
data = pd.read_csv('data/data.csv')

In [23]:
data.duplicated().sum()

706